In [16]:
using CSV, DataFrames
using Statistics, HypothesisTests
using Plots
using StatsPlots

In [17]:
df_x = DataFrame(CSV.File("weights/tract_X.csv"))[:, ["TRACT", "1"]]
rename!(df_x, [("1" => :x)]);

In [18]:
bhd = DataFrame(CSV.File("raw_data/cdc_places/PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2024_release_20250524.csv"))

m = [v == "Crude prevalence" for v in bhd[:, "Data_Value_Type"]]
bhd = bhd[m,:]

m = [v == 2022 for v in bhd[:, "Year"]]
bhd = bhd[m,:]

l = [
    "Frequent physical distress among adults",
    "Stroke among adults",
   "Feeling socially isolated among adults",
    "Fair or poor self-rated health status among adults",
    "All teeth lost among adults aged >=65 years",
    "Vision disability among adults",
    "Cognitive disability among adults",
    "Frequent mental distress among adults",
    "Lack of social and emotional support among adults",
    "Mobility disability among adults",
    "Current asthma among adults",
    "Chronic obstructive pulmonary disease among adults",
    "Coronary heart disease among adults",
    "Diagnosed diabetes among adults",
    "Hearing disability among adults",
    "Depression among adults",
    "Short sleep duration among adults",
    "Obesity among adults",
]

bhd_t = unique(bhd[:, [:LocationID]])
dropmissing!(bhd_t)

for (i, v) in enumerate(l)
    m = bhd[:, :Measure] .== v
    y = bhd[m, [:LocationID, :Data_Value]]
    y = combine(groupby(y, :LocationID), :Data_Value=>first=>:Data_Value)
    dropmissing!(y)
    bhd_t = leftjoin(bhd_t, y, on=:LocationID)
    rename!(bhd_t, [(:Data_Value => v)])
end

In [19]:
ha = DataFrame(CSV.File("raw_data/health-atlas-data_tract.csv"))

rename!(bhd_t, [(:LocationID => :TRACT)]);
rename!(ha, [(:GEOID => :TRACT)]);

res = outerjoin(outerjoin(bhd_t, df_x, on=:TRACT), ha, on=:TRACT);

In [20]:
df = DataFrame([l], ["BRFSS"])

Row,BRFSS
,String
1,Frequent physical distress among adults
2,Stroke among adults
3,Feeling socially isolated among adults
4,Fair or poor self-rated health status among adults
5,All teeth lost among adults aged >=65 years
6,Vision disability among adults
7,Cognitive disability among adults
8,Frequent mental distress among adults
9,Lack of social and emotional support among adults


In [21]:
res[:, "x"] = coalesce.(res[:, "x"], NaN)
vals = zeros(size(l))
for (i, v) in enumerate(l)
    res[:, v] = coalesce.(res[:, v], NaN)
    m = (isnan.(res[:, v]) .+ isnan.(res[:, "x"])) .== 0
    vals[i] = cor(res[m, "x"], res[m, v])
    println(cor(res[m, "x"], res[m, v]), "  ", v)
end

df[!, "x"] = vals;

0.7716375636316866  Frequent physical distress among adults
0.6888447770278404  Stroke among adults
0.39875415925619634  Feeling socially isolated among adults
0.7713226726796725  Fair or poor self-rated health status among adults
0.7215694242853932  All teeth lost among adults aged >=65 years
0.7196578989531719  Vision disability among adults
0.7022029355949845  Cognitive disability among adults
0.6175741411422876  Frequent mental distress among adults
0.4887271540862326  Lack of social and emotional support among adults
0.7561549641222425  Mobility disability among adults
0.5858439687710167  Current asthma among adults
0.659796532710566  Chronic obstructive pulmonary disease among adults
0.504165297358224  Coronary heart disease among adults
0.6938274316408716  Diagnosed diabetes among adults
0.4733376739254771  Hearing disability among adults
0.23421309318944825  Depression among adults
0.6381824144066294  Short sleep duration among adults
0.6772194218731781  Obesity among adults


In [22]:
res[:, "ndi"] = coalesce.(res[:, "ndi"], NaN)
vals = zeros(size(l))
for (i, v) in enumerate(l)
    res[:, v] = coalesce.(res[:, v], NaN)
    m = (isnan.(res[:, v]) .+ isnan.(res[:, "ndi"])) .== 0
    vals[i] = cor(res[m, "ndi"], res[m, v])
    println(cor(res[m, "ndi"], res[m, v]), "  ", v)
end

df[!, "ndi"] = vals;

0.7965873491574174  Frequent physical distress among adults
0.6334183837231552  Stroke among adults
0.4954948961759483  Feeling socially isolated among adults
0.8031984800360485  Fair or poor self-rated health status among adults
0.7516360589155675  All teeth lost among adults aged >=65 years
0.7441036664923635  Vision disability among adults
0.7875598049775906  Cognitive disability among adults
0.7102659108257876  Frequent mental distress among adults
0.559286721316524  Lack of social and emotional support among adults
0.7276346167359149  Mobility disability among adults
0.630787848456531  Current asthma among adults
0.6389760372811188  Chronic obstructive pulmonary disease among adults
0.42842197764680207  Coronary heart disease among adults
0.6531440025065457  Diagnosed diabetes among adults
0.4348022144207942  Hearing disability among adults
0.30858372899316194  Depression among adults
0.6615919021817607  Short sleep duration among adults
0.7350024884102914  Obesity among adults


In [23]:
res[:, "RPL_THEMES"] = coalesce.(res[:, "RPL_THEMES"], NaN)
vals = zeros(size(l))
for (i, v) in enumerate(l)
    res[:, v] = coalesce.(res[:, v], NaN)
    m = (isnan.(res[:, v]) .+ isnan.(res[:, "RPL_THEMES"])) .== 0
    vals[i] = cor(res[m, "RPL_THEMES"], res[m, v])
    println(cor(res[m, "RPL_THEMES"], res[m, v]), "  ", v)
end

df[!, "svi"] = vals;

0.6750545218949882  Frequent physical distress among adults
0.5197715515101056  Stroke among adults
0.5453618530454407  Feeling socially isolated among adults
0.745262067884984  Fair or poor self-rated health status among adults
0.6543346838090691  All teeth lost among adults aged >=65 years
0.7478470912726404  Vision disability among adults
0.6939059545204398  Cognitive disability among adults
0.596434759983643  Frequent mental distress among adults
0.653238921130044  Lack of social and emotional support among adults
0.6229603737667024  Mobility disability among adults
0.43248684209573346  Current asthma among adults
0.40421910144840434  Chronic obstructive pulmonary disease among adults
0.25696294258282576  Coronary heart disease among adults
0.5767041356441144  Diagnosed diabetes among adults
0.2731836568215937  Hearing disability among adults
0.10101387557076534  Depression among adults
0.5966469400260399  Short sleep duration among adults
0.46386550714176056  Obesity among adults


In [24]:
OneSampleTTest(df[!, "ndi"], df[!, "svi"] )

One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         0
    point estimate:          0.107847
    95% confidence interval: (0.06086, 0.1548)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0002

Details:
    number of observations:   18
    t-statistic:              4.842385617934543
    degrees of freedom:       17
    empirical standard error: 0.022271406667755914


In [25]:
OneSampleTTest(df[!, "x"], df[!, "svi"] )

One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         0
    point estimate:          0.0857654
    95% confidence interval: (0.02582, 0.1457)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0077

Details:
    number of observations:   18
    t-statistic:              3.018521654158261
    degrees of freedom:       17
    empirical standard error: 0.028413039401794314


In [26]:
OneSampleTTest(df[!, "x"], df[!, "ndi"] )

One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         0
    point estimate:          -0.0220814
    95% confidence interval: (-0.04884, 0.004675)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.0997

Details:
    number of observations:   18
    t-statistic:              -1.741199738247142
    degrees of freedom:       17
    empirical standard error: 0.012681695361713001


In [27]:
df[!, "x"] .- df[!, "ndi"]

18-element Vector{Float64}:
 -0.024949785525730794
  0.05542639330468524
 -0.09674073691975194
 -0.031875807356376074
 -0.030066634630174338
 -0.024445767539191587
 -0.08535686938260612
 -0.09269176968350001
 -0.07055956723029144
  0.028520347386327582
 -0.044943879685514276
  0.020820495429447194
  0.07574331971142195
  0.04068342913432588
  0.0385354595046829
 -0.0743706358037137
 -0.0234094877751313
 -0.05778306653711329

In [28]:
df[!, "ndi"] .- df[!, "svi"]

18-element Vector{Float64}:
  0.1215328272624292
  0.11364683221304961
 -0.04986695686949244
  0.0579364121510646
  0.09730137510649839
 -0.003743424780276894
  0.0936538504571508
  0.11383115084214457
 -0.09395219981351999
  0.10467424296921246
  0.19830100636079756
  0.2347569358327145
  0.1714590350639763
  0.07643986686243132
  0.16161855759920052
  0.2075698534223966
  0.06494496215572076
  0.2711369812685308

In [29]:
df[!, "x"] .- df[!, "svi"]

18-element Vector{Float64}:
  0.0965830417366984
  0.16907322551773485
 -0.14660769378924438
  0.026060604794688524
  0.06723474047632405
 -0.02818919231946848
  0.008296981074544685
  0.021139381158644555
 -0.16451176704381143
  0.13319459035554004
  0.15335712667528328
  0.2555774312621617
  0.24720235477539826
  0.11712329599675719
  0.20015401710388342
  0.13319921761868292
  0.041535474380589466
  0.21335391473141752

In [30]:
df

Row,BRFSS,x,ndi,svi
,String,Float64,Float64,Float64
1,Frequent physical distress among adults,0.771638,0.796587,0.675055
2,Stroke among adults,0.688845,0.633418,0.519772
3,Feeling socially isolated among adults,0.398754,0.495495,0.545362
4,Fair or poor self-rated health status among adults,0.771323,0.803198,0.745262
5,All teeth lost among adults aged >=65 years,0.721569,0.751636,0.654335
6,Vision disability among adults,0.719658,0.744104,0.747847
7,Cognitive disability among adults,0.702203,0.78756,0.693906
8,Frequent mental distress among adults,0.617574,0.710266,0.596435
9,Lack of social and emotional support among adults,0.488727,0.559287,0.653239
